In [ ]:
# Part 1: Implementing BPE from Scratch
# Step 1: Understanding the Data Structures
import re
from collections import defaultdict, Counter
def preprocess_text(text):
 """
 Lowercase and split text into words.
 """
 # Lowercase
 text = text.lower()
 # Split on whitespace and punctuation
 words = re.findall(r'\w+', text)
 return words

corpus = """
low low low lower lower lowest
the the the quick quick brown fox
"""
words = preprocess_text(corpus)
print("Words:", words)

#Count word frequencies
word_freqs = Counter(words)
print("Word frequencies:", word_freqs)

Words: ['low', 'low', 'low', 'lower', 'lower', 'lowest', 'the', 'the', 'the', 'quick', 'quick', 'brown', 'fox']
Word frequencies: Counter({'low': 3, 'the': 3, 'lower': 2, 'quick': 2, 'lowest': 1, 'brown': 1, 'fox': 1})


In [21]:
# Step 3: Initialize Character-Level Vocabulary
def get_vocab(word_freqs):
 """
 Split each word into characters + </w> marker.
 Returns dictionary: word -> list of tokens
 """
 vocab = {}
 for word, freq in word_freqs.items():
    # Split into characters, add </w> marker
    vocab[word] = list(word) + ['</w>']
 return vocab
vocab = get_vocab(word_freqs)
print("Initial vocabulary:")
for word, tokens in vocab.items():
 print(f" {word}: {tokens}")

Initial vocabulary:
 low: ['l', 'o', 'w', '</w>']
 lower: ['l', 'o', 'w', 'e', 'r', '</w>']
 lowest: ['l', 'o', 'w', 'e', 's', 't', '</w>']
 the: ['t', 'h', 'e', '</w>']
 quick: ['q', 'u', 'i', 'c', 'k', '</w>']
 brown: ['b', 'r', 'o', 'w', 'n', '</w>']
 fox: ['f', 'o', 'x', '</w>']


In [22]:
# Step 4: Count Pairs
def get_stats(vocab, word_freqs):
 """
 Count frequency of adjacent token pairs.
 """
 pairs = defaultdict(int)
 for word, freq in word_freqs.items():
    symbols = vocab[word]
    for i in range(len(symbols) - 1):
        pair = (symbols[i], symbols[i+1])
        pairs[pair] += freq
 return pairs

pairs = get_stats(vocab, word_freqs)
print("Pair frequencies:")
for pair, freq in sorted(pairs.items(), key=lambda x: -x[1])[:10]:
 print(f" {pair}: {freq}")

Pair frequencies:
 ('o', 'w'): 7
 ('l', 'o'): 6
 ('w', '</w>'): 3
 ('w', 'e'): 3
 ('t', 'h'): 3
 ('h', 'e'): 3
 ('e', '</w>'): 3
 ('e', 'r'): 2
 ('r', '</w>'): 2
 ('q', 'u'): 2


In [23]:
# Step 5: Merge Most Frequent Pair
def merge_vocab(pair, vocab):
 """
 Merge the given pair in the vocabulary.
 """
 new_vocab = {}

 # Create pattern to find the pair
 # Example: ('l', 'o') -> 'l o'
 pattern = ' '.join(pair)
 replacement = ''.join(pair)
 for word, tokens in vocab.items():
    # Convert tokens list to string for replacement
    tokens_str = ' '.join(tokens)
    # Replace first occurrence of pair
    tokens_str = tokens_str.replace(pattern, replacement)
    # Convert back to list
    new_vocab[word] = tokens_str.split()
 return new_vocab

most_frequent_pair = max(pairs, key=pairs.get)
print(f"\nMerging: {most_frequent_pair} → {''.join(most_frequent_pair)}")

vocab = merge_vocab(most_frequent_pair, vocab)
print("Vocabulary after merge:")
for word, tokens in vocab.items():
 print(f" {word}: {tokens}")


Merging: ('o', 'w') → ow
Vocabulary after merge:
 low: ['l', 'ow', '</w>']
 lower: ['l', 'ow', 'e', 'r', '</w>']
 lowest: ['l', 'ow', 'e', 's', 't', '</w>']
 the: ['t', 'h', 'e', '</w>']
 quick: ['q', 'u', 'i', 'c', 'k', '</w>']
 brown: ['b', 'r', 'ow', 'n', '</w>']
 fox: ['f', 'o', 'x', '</w>']


In [29]:
#Step 6: Complete BPE Training Loop
def train_bpe(word_freqs, num_merges):
 """
 Train BPE tokenizer for num_merges iterations.
 Returns vocabulary and list of merge operations.
 """
 vocab = get_vocab(word_freqs)
 merges = []
 for i in range(num_merges):
    pairs = get_stats(vocab, word_freqs)
    if not pairs:
        break

    # Find most frequent pair
    best_pair = max(pairs, key=pairs.get)

    # Merge it
    vocab = merge_vocab(best_pair, vocab)
    merges.append(best_pair)
    print(f"Iteration {i+1}: Merged {best_pair} → {''.join(best_pair)}(freq={pairs[best_pair]})")
 return vocab, merges

vocab, merges = train_bpe(word_freqs, num_merges=10)

print("\nFinal vocabulary:")
for word, tokens in vocab.items():
 print(f" {word}: {tokens}")
 
print("\nMerge operations (in order):")
for i, merge in enumerate(merges, 1):
 print(f" {i}. {merge[0]} + {merge[1]} → {''.join(merge)}")

Iteration 1: Merged ('o', 'w') → ow(freq=7)
Iteration 2: Merged ('l', 'ow') → low(freq=6)
Iteration 3: Merged ('low', '</w>') → low</w>(freq=3)
Iteration 4: Merged ('low', 'e') → lowe(freq=3)
Iteration 5: Merged ('t', 'h') → th(freq=3)
Iteration 6: Merged ('th', 'e') → the(freq=3)
Iteration 7: Merged ('the', '</w>') → the</w>(freq=3)
Iteration 8: Merged ('lowe', 'r') → lower(freq=2)
Iteration 9: Merged ('lower', '</w>') → lower</w>(freq=2)
Iteration 10: Merged ('q', 'u') → qu(freq=2)

Final vocabulary:
 low: ['low</w>']
 lower: ['lower</w>']
 lowest: ['lowe', 's', 't', '</w>']
 the: ['the</w>']
 quick: ['qu', 'i', 'c', 'k', '</w>']
 brown: ['b', 'r', 'ow', 'n', '</w>']
 fox: ['f', 'o', 'x', '</w>']

Merge operations (in order):
 1. o + w → ow
 2. l + ow → low
 3. low + </w> → low</w>
 4. low + e → lowe
 5. t + h → th
 6. th + e → the
 7. the + </w> → the</w>
 8. lowe + r → lower
 9. lower + </w> → lower</w>
 10. q + u → qu


In [25]:
# Step 7: BPE Encoding (Inference)
def encode_word(word, merges):
 """
 Encode a word using learned BPE merges.
 """
 # Start with character-level tokens
 tokens = list(word) + ['</w>']
 # Apply merges in order
 for merge in merges:
    pair = merge
    if pair not in zip(tokens[:-1], tokens[1:]):
       continue
 
    # Find and merge all occurrences
    i = 0
    while i < len(tokens) - 1:
        if (tokens[i], tokens[i+1]) == pair:
            # Merge
            tokens = tokens[:i] + [''.join(pair)] + tokens[i+2:]
        else:
            i += 1
    return toke

def encode_word_better(word, merges):
 """
 Encode a word by applying merges in order.
 
 """
 tokens = list(word) + ['</w>']
 
 for pair in merges:
    i = 0
    new_tokens = []
    while i < len(tokens):
        if i < len(tokens) - 1 and (tokens[i], tokens[i+1]) == pair:
            new_tokens.append(''.join(pair))
            i += 2
        else:
            new_tokens.append(tokens[i])
            i += 1
    tokens = new_tokens    
 return tokens

test_words = ['lower', 'lowest', 'newer']

print("\nEncoding test words:")
for word in test_words:
 encoded = encode_word_better(word, merges)
 print(f" {word} → {encoded}")



Encoding test words:
 lower → ['l', 'ow', 'e', 'r', '</w>']
 lowest → ['l', 'ow', 'e', 's', 't', '</w>']
 newer → ['n', 'e', 'w', 'e', 'r', '</w>']


In [26]:
# Step 8: Complete BPE Tokenizer Class
class SimpleBPETokenizer:
 def __init__(self):
    self.merges = []
    self.vocab = set()

 def train(self, text, num_merges):
    """Train BPE on text."""
    words = preprocess_text(text)
    word_freqs = Counter(words)

    vocab = get_vocab(word_freqs)
    
    # Extract initial vocabulary
    for tokens in vocab.values():
        self.vocab.update(tokens)
    
    # Perform merges
    for i in range(num_merges):
        pairs = get_stats(vocab, word_freqs)
        if not pairs:
            break

        best_pair = max(pairs, key=pairs.get)
        vocab = merge_vocab(best_pair, vocab)
        self.merges.append(best_pair)
        self.vocab.add(''.join(best_pair))

    print(f"Trained BPE with {len(self.merges)} merges")
    print(f"Vocabulary size: {len(self.vocab)}")

 def encode(self, text):
    """Encode text into BPE tokens."""
    words = preprocess_text(text)
    all_tokens = []
    for word in words:
        tokens = encode_word_better(word, self.merges)
        all_tokens.extend(tokens)
    return all_tokens

 def get_vocab_size(self):
    return len(self.vocab)

tokenizer = SimpleBPETokenizer()
tokenizer.train(corpus, num_merges=15)

test_text = "the quick brown fox jumps lower"
tokens = tokenizer.encode(test_text)
print(f"\nEncoded: {test_text}")
print(f"Tokens: {tokens}")
print(f"Token count: {len(tokens)}")

Trained BPE with 15 merges
Vocabulary size: 33

Encoded: the quick brown fox jumps lower
Tokens: ['the</w>', 'quick</w>', 'b', 'r', 'ow', 'n', '</w>', 'f', 'o', 'x', '</w>', 'j', 'u', 'm', 'p', 's', '</w>', 'lower</w>']
Token count: 18


In [28]:
# Part 2: Using Existing Libraries
# Using Hugging Face Transformers for WordPiece (BERT)

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "Playing with BERT tokenization is fun!"

tokens = tokenizer.tokenize(text)
print("BERT WordPiece tokens:", tokens)

token_ids = tokenizer.encode(text)
print("Token IDs:", token_ids)

decoded = tokenizer.decode(token_ids)
print("Decoded:", decoded)

print(f"BERT vocab size: {tokenizer.vocab_size}")

c:\Users\prosi\Documents\GitHub\NLP_Course\envire\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\prosi\Documents\GitHub\NLP_Course\envire\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prosi\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitat

BERT WordPiece tokens: ['playing', 'with', 'bert', 'token', '##ization', 'is', 'fun', '!']
Token IDs: [101, 2652, 2007, 14324, 19204, 3989, 2003, 4569, 999, 102]
Decoded: [CLS] playing with bert tokenization is fun! [SEP]
BERT vocab size: 30522


In [38]:
#Using SentencePiece
import sentencepiece as spm
import os

training_text = """
The quick brown fox jumps over the lazy dog.
Natural language processing is fascinating.
Tokenization is the first step in NLP.
Subword tokenization solves the OOV problem.
BPE, WordPiece, and SentencePiece are popular methods.
Machine learning models need tokenized input.
""" * 100 # Repeat for more data

with open('train.txt', 'w') as f:
 f.write(training_text)

spm.SentencePieceTrainer.train(
 input='train.txt',
 model_prefix='sp_model',
 vocab_size=300,
 model_type='bpe', # or 'unigram'
 character_coverage=1.0,
 pad_id=0,
 unk_id=1,
 bos_id=2,
 eos_id=3
)

print("SentencePiece model trained!")

sp = spm.SentencePieceProcessor()
sp.load('sp_model.model')

test_text = "Natural language processing"
tokens = sp.encode_as_pieces(test_text)
print(f"\nSentencePiece tokens: {tokens}")

ids = sp.encode_as_ids(test_text)
print(f"Token IDs: {ids}")

decoded = sp.decode_pieces(tokens)
print(f"Decoded: {decoded}")

decoded_ids = sp.decode_ids(ids)
print(f"Decoded from IDs: {decoded_ids}")

print(f"SentencePiece vocab size: {sp.vocab_size()}")

print("\nFirst 20 tokens in vocabulary:")
for i in range(20):
 print(f" {i}: {sp.id_to_piece(i)}")

spm.SentencePieceTrainer.train(
 input='train.txt',
 model_prefix='sp_unigram',
 vocab_size=80,
 model_type='unigram', # Unigram LM
 character_coverage=1.0
)
sp_unigram = spm.SentencePieceProcessor()
sp_unigram.load('sp_unigram.model')

test_text = "tokenization strategies"
bpe_tokens = sp.encode_as_pieces(test_text)

unigram_tokens = sp_unigram.encode_as_pieces(test_text)
print(f"Text: {test_text}")
print(f"BPE: {bpe_tokens}")
print(f"Unigram: {unigram_tokens}")

SentencePiece model trained!

SentencePiece tokens: ['▁Natural', '▁language', '▁processing']
Token IDs: [146, 153, 157]
Decoded: Natural language processing
Decoded from IDs: Natural language processing
SentencePiece vocab size: 300

First 20 tokens in vocabulary:
 0: <pad>
 1: <unk>
 2: <s>
 3: </s>
 4: in
 5: en
 6: ▁t
 7: at
 8: ce
 9: he
 10: ar
 11: iz
 12: ok
 13: ro
 14: ▁f
 15: ▁l
 16: ▁p
 17: ing
 18: eniz
 19: ▁the
Text: tokenization strategies
BPE: ['▁tokenization', '▁st', 'r', 'at', 'e', 'g', 'i', 'es']
Unigram: ['▁tokeniz', 'ati', 'o', 'n', '▁', 'st', 'r', 'at', 'e', 'g', 'i', 'e', 's']


In [42]:
#Part 3: Comparison and Analysis

test_sentences = [
 "The quick brown fox jumps over the lazy dog.",
 "Natural language processing is fascinating!",
 "Subword tokenization: BPE, WordPiece, SentencePiece.",
 "COVID-19 pandemic affected the world in 2020.",
 "Machine learning models require tokenized input.",
 "Typo example: recieve instead of receive.",
 "Scientific term: deoxyribonucleic acid (DNA)."
]

print("=" * 80)
print("TOKENIZATION COMPARISON")
print("=" * 80)

for sent in test_sentences:
 print(f"\nOriginal: {sent}")
 print("-" * 80)

 # Our simple BPE
 simple_tokens = tokenizer.encode(sent)
 print(f"Simple BPE: {simple_tokens[:15]}...") # First 15 tokens
 print(f" Token count: {len(simple_tokens)}")
 
 # BERT WordPiece
 bert_tokens = BertTokenizer.from_pretrained('bert-base-uncased').tokenize(sent)
 print(f"BERT WordPiece: {bert_tokens}")
 print(f" Token count: {len(bert_tokens)}")
 
 # SentencePiece BPE
 sp_tokens = sp.encode_as_pieces(sent)
 print(f"SentencePiece: {sp_tokens}")
 print(f" Token count: {len(sp_tokens)}")






TOKENIZATION COMPARISON

Original: The quick brown fox jumps over the lazy dog.
--------------------------------------------------------------------------------
Simple BPE: [101, 1996, 4248, 2829, 4419, 14523, 2058, 1996, 13971, 3899, 1012, 102]...
 Token count: 12
BERT WordPiece: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
 Token count: 10
SentencePiece: ['▁The', '▁quick', '▁brown', '▁fox', '▁jumps', '▁over', '▁the', '▁lazy', '▁dog', '.']
 Token count: 10

Original: Natural language processing is fascinating!
--------------------------------------------------------------------------------
Simple BPE: [101, 3019, 2653, 6364, 2003, 17160, 999, 102]...
 Token count: 8
BERT WordPiece: ['natural', 'language', 'processing', 'is', 'fascinating', '!']
 Token count: 6
SentencePiece: ['▁Natural', '▁language', '▁processing', '▁is', '▁fascinating', '!']
 Token count: 6

Original: Subword tokenization: BPE, WordPiece, SentencePiece.
--------------------------------

In [47]:
def calculate_fertility(text, tokenizer_func):
 """
 Calculate fertility: avg tokens per word.
 """
 words = text.split()
 tokens = tokenizer_func(text)

 # Remove special tokens
 tokens = [t for t in tokens if t not in ['<s>', '</s>', '[CLS]', '[SEP]', '<pad>']]
 fertility = len(tokens) / len(words) if words else 0
 return fertility, len(words), len(tokens)

print("\n" + "=" * 80)
print("FERTILITY COMPARISON")
print("=" * 80)

for sent in test_sentences:
 print(f"\nSentence: {sent[:50]}...")

 # BERT
 bert_fert, words, bert_tokens = calculate_fertility(sent, lambda t: BertTokenizer.from_pretrained('bert-base-uncased').tokenize(t))

 # SentencePiece
 sp_fert, _, sp_tokens = calculate_fertility(sent,lambda t: sp.encode_as_pieces(t))

 print(f" Words: {words}")
 print(f" BERT WordPiece: {bert_tokens} tokens → fertility = {bert_fert:.2f}")
 print(f" SentencePiece: {sp_tokens} tokens → fertility = {sp_fert:.2f}")


FERTILITY COMPARISON

Sentence: The quick brown fox jumps over the lazy dog....
 Words: 9
 BERT WordPiece: 10 tokens → fertility = 1.11
 SentencePiece: 10 tokens → fertility = 1.11

Sentence: Natural language processing is fascinating!...
 Words: 5
 BERT WordPiece: 6 tokens → fertility = 1.20
 SentencePiece: 6 tokens → fertility = 1.20

Sentence: Subword tokenization: BPE, WordPiece, SentencePiec...
 Words: 5
 BERT WordPiece: 14 tokens → fertility = 2.80
 SentencePiece: 9 tokens → fertility = 1.80

Sentence: COVID-19 pandemic affected the world in 2020....
 Words: 7
 BERT WordPiece: 13 tokens → fertility = 1.86
 SentencePiece: 24 tokens → fertility = 3.43

Sentence: Machine learning models require tokenized input....
 Words: 6
 BERT WordPiece: 8 tokens → fertility = 1.33
 SentencePiece: 11 tokens → fertility = 1.83

Sentence: Typo example: recieve instead of receive....
 Words: 6
 BERT WordPiece: 11 tokens → fertility = 1.83
 SentencePiece: 27 tokens → fertility = 4.50

Sentence: Scie

In [54]:
def analyze_vocabulary_coverage(text, tokenizer, unk_token):
 """
 Calculate percentage of unknown tokens.
 """
 tokens = tokenizer.tokenize(text)
 unk_count = tokens.count(unk_token)
 total = len(tokens)
 coverage = (1 - unk_count / total) * 100 if total > 0 else 0
 return coverage, unk_count, total

medical_text = """
Azithromycin is a macrolide antibiotic used to treat pneumonia.
Deoxyribonucleic acid stores genetic information in chromosomes.
"""
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
print("\n" + "=" * 80)
print("OUT-OF-DOMAIN VOCABULARY COVERAGE")
print("=" * 80)
print(f"Text: {medical_text[:100]}...")
bert_cov, bert_unk, bert_total = analyze_vocabulary_coverage(medical_text, BertTokenizer.from_pretrained('bert-base-uncased'),'[UNK]')
print(f"\nBERT WordPiece:")
print(f" Coverage: {bert_cov:.1f}%")
print(f" Unknown tokens: {bert_unk}/{bert_total}")




OUT-OF-DOMAIN VOCABULARY COVERAGE
Text: 
Azithromycin is a macrolide antibiotic used to treat pneumonia.
Deoxyribonucleic acid stores geneti...

BERT WordPiece:
 Coverage: 100.0%
 Unknown tokens: 0/31


In [58]:
#Timing Comparison
import time
def benchmark_tokenizer(text, tokenize_func, num_runs=100):
 """
 Benchmark tokenization speed.
 """
 start = time.time()
 for _ in range(num_runs):
    tokens = tokenize_func(text)
 end = time.time()

 avg_time = (end - start) / num_runs * 1000 # milliseconds
 return avg_time

long_text = " ".join(test_sentences) * 10

print("\n" + "=" * 80)
print("TOKENIZATION SPEED COMPARISON")
print("=" * 80)
print(f"Text length: {len(long_text)} characters\n")

bert_time = benchmark_tokenizer(
 long_text,
 lambda t: BertTokenizer.from_pretrained('bert-base-uncased').tokenize(t)
)
sp_time = benchmark_tokenizer(
 long_text,
 lambda t: sp.encode_as_pieces(t)
)
print(f"BERT WordPiece: {bert_time:.2f} ms/run")
print(f"SentencePiece: {sp_time:.2f} ms/run")
print(f"Speedup: {bert_time/sp_time:.2f}x")



TOKENIZATION SPEED COMPARISON
Text length: 3240 characters



Token indices sequence length is longer than the specified maximum sequence length for this model (780 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (780 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (780 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (780 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (780 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

BERT WordPiece: 819.31 ms/run
SentencePiece: 0.83 ms/run
Speedup: 987.72x
